##  Automating Microsoft Office with Python

About Me: Nikhil Kabbin
- Data Scientist @Gramener
- Twitter/Github: @nikhilkabbin

<img width="1080" src="comic.png">

Windows applications, for many years, have provided a COM API for automation. This includes Microsoft Office as well.

pywin32 is a library that lets you do many interesting things in Windows, including access these COM APIs.

For example, to open PowerPoint and draw a circle, this is what it takes:

In [29]:
import win32com.client

# Open PowerPoint
Application = win32com.client.Dispatch("PowerPoint.Application")

# Add a presentation
Presentation = Application.Presentations.Add()

# Add a slide with a blank layout (12 stands for blank layout)
Base = Presentation.Slides.Add(1, 12)


# Add an oval. Shape 9 is an oval.
oval = Base.Shapes.AddShape(9, 100, 100, 100, 100)

You'll have to try this out to see the result, but just FYI, this will open a new PowerPoint window and add a slide with a circle in it.

This opens up a lot of opportunities for slideware. Similarly, we can open an Excel application, add a circle, and change a few cells.

### We can also open Excel application and add a circle, and change few cells

In [30]:
# Open Excel
Application = win32com.client.Dispatch("Excel.Application")

# Show Excel. Unlike PPT, Word & Excel open up "hidden"
Application.Visible = 10

# Add a workbook
Workbook = Application.Workbooks.Add()

# Take the active sheet
Base = Workbook.ActiveSheet

# Add an oval. Shape 9 is an oval.
oval = Base.Shapes.AddShape(9, 100, 100, 100, 100)

# In the first row, add Values: 0.0, 0.5, 1.0
Base.Cells(1, 1).Value = 'Values'
Base.Cells(1, 2).Value = 0.0
Base.Cells(1, 3).Value = 0.5
Base.Cells(1, 4).Value = 1.0

This means one can go about creating Excel models directly with Python.

## Picturing the IMDb Top 250

Let's begin by creating a slide that shows all of the [Top 250 movies](http://www.imdb.com/chart/top) on the IMDb.

First, let's load all the movies.

In [33]:
from bs4 import BeautifulSoup as bs
import requests
url = 'http://www.imdb.com/chart/top'

r = requests.get(url)
soup = bs(r.content, "html.parser")
tds = soup.findAll('td', {'class': 'titleColumn'})

movies = []
for td in tds:
    movies.append(td.findChildren('a')[0])

movies
# print("Movie Name: {}".format(movies[0].text))
# print("Movies Length: {}".format(len(movies)))

[<a href="/title/tt0111161/" title="Frank Darabont (dir.), Tim Robbins, Morgan Freeman">The Shawshank Redemption</a>,
 <a href="/title/tt0068646/" title="Francis Ford Coppola (dir.), Marlon Brando, Al Pacino">The Godfather</a>,
 <a href="/title/tt0071562/" title="Francis Ford Coppola (dir.), Al Pacino, Robert De Niro">The Godfather: Part II</a>,
 <a href="/title/tt0468569/" title="Christopher Nolan (dir.), Christian Bale, Heath Ledger">The Dark Knight</a>,
 <a href="/title/tt0050083/" title="Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb">12 Angry Men</a>,
 <a href="/title/tt0108052/" title="Steven Spielberg (dir.), Liam Neeson, Ralph Fiennes">Schindler's List</a>,
 <a href="/title/tt0167260/" title="Peter Jackson (dir.), Elijah Wood, Viggo Mortensen">The Lord of the Rings: The Return of the King</a>,
 <a href="/title/tt0110912/" title="Quentin Tarantino (dir.), John Travolta, Uma Thurman">Pulp Fiction</a>,
 <a href="/title/tt0060196/" title="Sergio Leone (dir.), Clint Eastwood, Eli Wal

We can show these movies. But before that, we can't remember these numbers for the shapes (like 9 is for circle). Let's pre-define those in line with how Office uses them and import them.

In [34]:
from MSO import *

Let's draw each movie poster as a little box on a 25x10 grid. We don't have the images yet, but first, let's just draw the rectangles.

In [36]:
import win32com.client

# Open PowerPoint
Application = win32com.client.Dispatch("PowerPoint.Application")

# Add a presentation
Presentation = Application.Presentations.Add()

Presentation.PageSetup.SlideSize = ppSlideSizeOnScreen

Base = Presentation.Slides.Add(1, ppLayoutBlank)


width, height = 28, 41
for i, movie in enumerate(movies):
    x = 10 + width * (i % 25)
    y = 100 + height * (i // 25)
    r = Base.Shapes.AddShape(
            msoShapeRectangle,
            x, y,
            width, height)

It would be nice to get posters into those, so let's scrape the posters.

In [37]:
import os
from urllib.parse import urljoin
from urllib.request import urlretrieve
from hashlib import md5

if not os.path.exists('img'):
    os.makedirs('img')
    
def filename(movie):
    '''Filename = MD5 hash of its title in UTF8'''
    name = md5(movie.text.encode('utf8')).hexdigest()
    return os.path.join('img', name + '.jpg')

count = 0
for movie in movies:
    if os.path.exists(filename(movie)):
        continue
    count += 1
        
    url = urljoin('http://www.imdb.com/', movie['href'])
    ru = requests.get(url)
    usoup = bs(ru.content, "html.parser")
    img = usoup.find('div', {'class': 'poster'}).findChildren('img')[0]
    urlretrieve(img.get('src'), filename(movie))

Now, instead of just rectangles, we'll use the posters.

In [40]:
# Open PowerPoint
Application = win32com.client.Dispatch("PowerPoint.Application")

# Add a presentation
Presentation = Application.Presentations.Add()
Presentation.PageSetup.SlideSize = ppSlideSizeOnScreen

Base = Presentation.Slides.Add(1, ppLayoutBlank)

width, height = 28, 41
for i, movie in enumerate(movies):
    x = 10 + width * (i % 25)
    y = 100 + height * (i // 25)
    image = Base.Shapes.AddPicture(
        os.path.abspath(filename(movie)),
        LinkToFile=True,
        SaveWithDocument=False,
        Left=x, Top=y,
        Width=width, Height=height)

Wouldn't it be nice to have these hyperlinked to the movies?

In [43]:
# Open PowerPoint
Application = win32com.client.Dispatch("PowerPoint.Application")

# Add a presentation
Presentation = Application.Presentations.Add()
Presentation.PageSetup.SlideSize = ppSlideSizeOnScreen

Base = Presentation.Slides.Add(1, ppLayoutBlank)

width, height = 28, 41
for i, movie in enumerate(movies):
    x = 10 + width * (i % 25)
    y = 100 + height * (i // 25)
    image = Base.Shapes.AddPicture(
        os.path.abspath(filename(movie)),
        LinkToFile=True,
        SaveWithDocument=False,
        Left=x, Top=y,
        Width=width, Height=height)
    url = urljoin('http://www.imdb.com/', movie.get('href'))
    link = image.ActionSettings(ppMouseClick).Hyperlink
    link.Address = url
    link.ScreenTip = movie.text # .encode('cp1252')

This is ordered by rank, which is useful, but this makes it hard to locate a specific movie. What if we could sort this alphabetically?

But then, we don't want to lose the ordering by rank either. Could we, perhaps, get these movies to move on the click of a button to alphabetical or rank order?

Let's start by adding two buttons -- one to sort alphabetically and the ohter to sort by rank.

In [44]:
Base = Presentation.Slides.Add(1, ppLayoutBlank)

# Add two buttons: alphabetical and by rating
button_alpha = Base.Shapes.AddShape(msoShapeRectangle, 400, 10, 150, 40)
button_alpha.TextFrame.TextRange.Text = 'Alphabetical'

button_rating = Base.Shapes.AddShape(msoShapeRectangle, 560, 10, 150, 40)
button_rating.TextFrame.TextRange.Text = 'By rating'

# Get the index position when sorted alphabetically
movies_alpha = sorted(movies, key=lambda v: v.text)
index_alpha = dict((movie.text, i) for i, movie in enumerate(movies_alpha))

We'll create a function that moves an image along a path when a trigger is clicked. This will be applied to each of the images.

In [45]:
def animate(seq, image, trigger, path, duration=1.5):
    '''Move image along path when trigger is clicked'''
    effect = seq.AddEffect(
       Shape=image,
       effectId=msoAnimEffectPathDown,
       trigger=msoAnimTriggerOnShapeClick,
    )
    ani = effect.Behaviors.Add(msoAnimTypeMotion)
    ani.MotionEffect.Path = path
    effect.Timing.TriggerType = msoAnimTriggerWithPrevious
    effect.Timing.TriggerShape = trigger
    effect.Timing.Duration = duration

In [46]:
seq_alpha = Base.TimeLine.InteractiveSequences.Add()
seq_rating = Base.TimeLine.InteractiveSequences.Add()

width, height = 28, 41
for i, movie in enumerate(movies):
    x = 10 + width * (i % 25)
    y = 100 + height * (i // 25)
    image = Base.Shapes.AddPicture(
        os.path.abspath(filename(movie)),
        LinkToFile=True,
        SaveWithDocument=False,
        Left=x, Top=y,
        Width=width, Height=height)
    url = urljoin('http://www.imdb.com/', movie['href'])
    link = image.ActionSettings(ppMouseClick).Hyperlink
    link.Address = url
    link.ScreenTip = movie.text  # .encode('cp1252')
    
    # Alphabetical 
    index = index_alpha[movie.text]
    animate(seq_alpha, image, trigger=button_alpha, path='M0,0 L{:.3f},{:.3f}'.format(
        (10 + width * (index % 25) - x) / 720.,
        (100 + height * (index // 25) - y) / 540.,
    ))
    
    # By rating
    animate(seq_rating, image, trigger=button_rating, path='M{:.3f},{:.3f} L0,0'.format(
        (10 + width * (index % 25) - x) / 720.,
        (100 + height * (index // 25) - y) / 540.,
    ))

# Real-life examples

You can see how these were put to use at [Gramener](http://gramener.com/):

- [Vijay Karnataka's election coverage](http://gramener.com/vijaykarnataka/) was produced entirely in PowerPoint using Python. (And in Kannada, no less!)
- [Appstore downloads](http://gramener.com/appstore/appstore.pptx) -- a mini interactive application that shows the number of downloads from an app store, broken up by country, device, customer segment and time.
- [Revenue breakup](http://gramener.com/fmcg/fmcg-revenue-breakup.pptx) of an FMCG company as a clickable application

#### This is Fancy, but... It will probably take lot of time to create these charts.

### What if I want to automate existing charts in Office?

### Microsoft adopted XML for PowerPoint 2007 and its other Office programs

In [48]:
from pptx import Presentation

prs = Presentation()
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]

title.text = "Hello, World!"
subtitle.text = "Subtitle"

prs.save('Azure.pptx')

Now let's try to add a simple bar chart in our slide!

In [49]:
from pptx import Presentation
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches

# create presentation with 1 slide ------
prs = Presentation()
slide = prs.slides.add_slide(prs.slide_layouts[5])

# define chart data ---------------------
chart_data = CategoryChartData()
chart_data.categories = ['East', 'West', 'Midwest']
chart_data.add_series('Series 1', (19.2, 21.4, 16.7))

# add chart to slide --------------------
x, y, cx, cy = Inches(2), Inches(2), Inches(6), Inches(4.5)
slide.shapes.add_chart(
    XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data
)

prs.save('charttest.pptx')

#### This is great! But I still need to write lot of code. Can we spend less time in writing code and focus on Chart and slides?

#### [Gramex PPTGen](https://learn.gramener.com/guide/pptxhandler/#pptgen-command-line) will help us do that! 

In [51]:
from gramex.pptgen import pptgen
pptgen(
  source='input.pptx',      # optional path to source. Defaults to blank PPT with 1 slide
  target='output2.pptx',     # optional target. Otherwise, returns the pptx.Presentation()
  change={                  # Configurations are same as when loading from the YAML file
    'Title 1': {            # Take the shape named "Title 1"
      'text': 'Welcome to Azure DataFest 2019!'    # Replace its text with "New Title"
    }
  }
)

#### Or You can use Command Line interface...

```pptgen config.yaml --target new.pptx "--change.Title 1.text" "Updated title"```

In [17]:
### TODO: pptgen chart example

# Live tweeting

Just for kicks, let's use PowerPoint as a dashboard to show live tweets.

I picked [TwitterAPI](https://github.com/geduldig/TwitterAPI) to get streaming results, but [twython](https://github.com/ryanmcgrath/twython) and [Python Twitter Tools](https://github.com/sixohsix/twitter) look fine too.

In [52]:
from TwitterAPI import TwitterAPI

# I'm keeping my keys and secrets in a secret file.
from secret_twitter import consumer_key, consumer_secret, access_token_key, access_token_secret
api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret)

In [53]:
def draw_tweet(Base, item, pos):
    y = 40 + (pos % 4) * 120
    
    image = Base.Shapes.AddPicture(
        # To get the larger resolution image, just remove _normal from the URL
        item['user']['profile_image_url'].replace('_normal', ''),
        LinkToFile=True,
        SaveWithDocument=False,
        Left=20, Top=y,
        Width=100, Height=100)
    
    try:
        status = item['text']  #.encode('cp1252')
    except UnicodeEncodeError:
        status = item['text']
    text = Base.Shapes.AddShape(msoShapeRectangle, 130, y, 460, 100)
    text.Fill.ForeColor.ObjectThemeColor = msoThemeColorText1
    text.Fill.ForeColor.Brightness = +0.95
    text.Line.Visible = msoFalse
    text.TextFrame.TextRange.Text = status
    text.TextFrame.TextRange.Font.Color.ObjectThemeColor = msoThemeColorText1
    text.TextFrame.TextRange.ParagraphFormat.Alignment = ppAlignLeft
    
    user = Base.Shapes.AddShape(msoShapeRectangle, 600, y, 100, 100)
    user.Fill.ForeColor.ObjectThemeColor = msoThemeColorAccent6
    user.Line.Visible = False
    user.TextFrame.TextRange.Text = '@' + item['user']['screen_name']

In [54]:
# Open PowerPoint
Application = win32com.client.Dispatch("PowerPoint.Application")

# Add a presentation
Presentation = Application.Presentations.Add()
Presentation.PageSetup.SlideSize = ppSlideSizeOnScreen

Base = Presentation.Slides.Add(1, ppLayoutBlank)
r = api.request('search/tweets', {'q':'azuredatafest'})

for pos, item in enumerate(r):
    draw_tweet(Base, item, pos)
    if pos > 10:
        break

## Thank You :-)